Dans cette partie, on a fait l'analyse descriptive sur le taux de réussite du bac général et du bac technique.

# Partie 0: Traitement des données et libraries

In [55]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import datetime as dt
import seaborn as sns
import plotly.express as px
%matplotlib inline
import sys
import warnings
from  matplotlib.colors import LinearSegmentedColormap
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [56]:
# Base de données pour l'analyse de Brevet
#SOURCE:https://data.education.gouv.fr/explore/dataset/fr-en-dnb-par-etablissement/information/?disjunctive.session&disjunctive.numero_d_etablissement&disjunctive.denomination_principale&disjunctive.secteur_d_enseignement&disjunctive.commune_et_arrondissement&disjunctive.commune_et_arrondissement_lib_l&disjunctive.departement&disjunctive.departement_libelle&disjunctive.academie&disjunctive.academie_libelle&disjunctive.region&disjunctive.region_libelle
df_brevet=pd.read_csv("fr-en-dnb-par-etablissement.csv",  encoding='utf-8-sig', sep=';',na_values=["-"])

In [57]:
#Traiter les données pour transformer en valeur flotante
df_brevet['taux_de_reussite']=df_brevet['taux_de_reussite'].replace('[\%,]', '', regex=True).astype(float)/100

In [58]:
#Convertir automatiquement les données 
df_brevet.convert_dtypes(infer_objects=True, convert_string=True,
      convert_integer=True, convert_boolean=True, convert_floating=True)

,session,numero_d_etablissement,denomination_principale,patronyme,secteur_d_enseignement,commune_et_arrondissement,commune_et_arrondissement_lib_l,departement,departement_libelle,academie,...,region,region_libelle,nombre_d_inscrits,nombre_de_presents,nombre_total_d_admis,nombre_d_admis_sans_mention,nombre_d_admis_mention_ab,nombre_d_admis_mention_b,nombre_d_admis_mention_tb,taux_de_reussite
0,2007,0240106K,COLLEGE,<NA>,PUBLIC,24568,VELINES,024,DORDOGNE,4,...,15,NOUVELLE-AQUITAINE,47,46,41,21,13,4,3,89.1
1,2007,0240158S,COLLEGE,JEAN MONNET,PUBLIC,24223,LALINDE,024,DORDOGNE,4,...,15,NOUVELLE-AQUITAINE,85,84,70,37,25,7,1,83.3
2,2007,0240650B,COLLEGE,ARTHUR RIMBAUD,PUBLIC,24372,SAINT-ASTIER,024,DORDOGNE,4,...,15,NOUVELLE-AQUITAINE,109,108,105,52,34,16,3,97.2
3,2007,0241033T,LYCEE PROFESSIONNEL,LE CLUZEAU,PRIVE,24534,SIGOULES-ET-FLAUGEAC,024,DORDOGNE,4,...,15,NOUVELLE-AQUITAINE,39,39,38,13,18,7,0,97.4
4,2007,0241042C,COLLEGE,BERTRAN DE BORN,PUBLIC,24322,PERIGUEUX,024,DORDOGNE,4,...,15,NOUVELLE-AQUITAINE,122,121,106,34,32,24,16,87.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130759,2015,0300039E,COLLEGE,DE LA GALABERTE,PUBLIC,30263,SAINT-HIPPOLYTE-DU-FORT,030,GARD,11,...,16,OCCITANIE,69,69,56,20,17,12,7,81.1
130760,2015,0300050S,COLLEGE,<NA>,PUBLIC,30344,VERGEZE,030,GARD,11,...,16,OCCITANIE,151,151,129,44,37,28,20,85.4
130761,2015,0300922P,COLLEGE,TAISSON,PRIVE,30007,ALES,030,GARD,11,...,16,OCCITANIE,109,109,107,48,34,20,5,98.1
130762,2015,0300949U,COLLEGE,JEAN RACINE,PUBLIC,30007,ALES,030,GARD,11,...,16,OCCITANIE,120,117,105,35,34,24,12,89.7


In [59]:
#Convertir manuellement les donnees en datetime et object
df_brevet['session']=pd.to_datetime(df_brevet['session'], format= '%Y')
df_brevet['academie']=df_brevet['academie'].astype(str)
df_brevet['region']=df_brevet['region'].astype(str)
df_brevet['session'] = pd.DatetimeIndex(df_brevet['session']).year

In [60]:
#Sélectionner les départements de la petite couronne
df1_brevet=df_brevet[df_brevet['departement'].isin(['075','092','093','094'])]

In [94]:
# Base de données pour l'analyse de BAC
#SOURCE:https://data.education.gouv.fr/explore/dataset/fr-en-indicateurs-de-resultat-des-lycees-denseignement-general-et-technologique/table/
df_bac=pd.read_csv('fr-en-indicateurs-de-resultat-des-lycees-denseignement-general-et-technologique.csv',   encoding='utf-8-sig', sep=';')

In [63]:
#Convertir automatiquement les données 
df_bac.convert_dtypes(infer_objects=True, convert_string=True,
      convert_integer=True, convert_boolean=True, convert_floating=True)

,Etablissement,Annee,Ville,UAI,Code commune,Academie,Departement,Secteur,Presents - L,Presents - ES,...,Valeur ajoutee du taux de mentions - L,Valeur ajoutee du taux de mentions - ES,Valeur ajoutee du taux de mentions - S,Valeur ajoutee du taux de mentions - STI2D,Valeur ajoutee du taux de mentions - STD2A,Valeur ajoutee du taux de mentions - STMG,Valeur ajoutee du taux de mentions - STL,Valeur ajoutee du taux de mentions - ST2S,Valeur ajoutee du taux de mentions - TMD,Valeur ajoutee du taux de mentions - STHR
0,LYCEE LALANDE,2019,BOURG EN BRESSE,0010013J,1053,LYON,Ain,PU,54,78,...,-3,4,-2,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
1,LYCEE JOSEPH-MARIE CARRIAT (GENERAL ET TECHNO.),2019,BOURG EN BRESSE,0010016M,1053,LYON,Ain,PU,<NA>,51,...,<NA>,-14,0,-15,NaN,-21,<NA>,<NA>,NaN,NaN
2,LYCEE INTERNATIONAL (GENERAL ET TECHNO.),2019,FERNEY VOLTAIRE,0010072Y,1160,LYON,Ain,PU,42,200,...,-2,-7,-8,<NA>,NaN,-17,<NA>,<NA>,NaN,NaN
3,LYCEE JEANNE D'ARC,2019,CESSY,0010878Z,1071,LYON,Ain,PR,19,39,...,-5,-15,15,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
4,LYCEE LA PROVIDENCE,2019,LAON,0021872Z,2408,AMIENS,Aisne,PR,3,18,...,ND,-29,-40,<NA>,NaN,-21,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20838,LYCEE DE VINCENDO (GENERAL ET TECHNO.),2020,ST JOSEPH,9741230U,97412,LA REUNION,La R√©union,PU,20,47,...,-5,17,12,<NA>,NaN,6,<NA>,3,NaN,NaN
20839,LYCEE GUSTAVE EIFFEL (GENERAL ET TECHNO.),2020,OUANGANI,9760125G,97614,MAYOTTE,Mayotte,PU,33,34,...,ND,ND,ND,ND,NaN,<NA>,<NA>,<NA>,NaN,NaN
20840,LYCEE DE SADA (GENERAL ET TECHNO.),2020,SADA,9760182U,97616,MAYOTTE,Mayotte,PU,70,98,...,ND,ND,ND,<NA>,NaN,ND,<NA>,<NA>,NaN,NaN
20841,LYCEE DE DEMBENI (GENERAL ET TECHNO.),2020,DEMBENI,9760338N,97607,MAYOTTE,Mayotte,PU,70,68,...,ND,ND,ND,ND,NaN,<NA>,ND,ND,NaN,NaN


In [64]:
#EXPLIQUER:
df_bac['total_eleves_general']=df_bac['Presents - L']+df_bac['Presents - ES']+df_bac['Presents - S']
df_bac['Taux_reussite_general']=(df_bac['Presents - L']*df_bac['Taux de reussite - L']+df_bac['Presents - S']*df_bac['Taux de reussite - S']+df_bac['Presents - ES']*df_bac['Taux de reussite - ES'])/df_bac['total_eleves_general']
df_bac['Taux_mentions_general']=(df_bac['Presents - L']*df_bac['Taux de mentions - L']+df_bac['Presents - S']*df_bac['Taux de mentions - S']+df_bac['Presents - ES']*df_bac['Taux de mentions - ES'])/df_bac['total_eleves_general']

df_ayuda=df_bac[[
 'Presents - STG',
 'Presents - STI2D',
 'Presents - STD2A',
 'Presents - STMG',
 'Presents - STI',
 'Presents - STL',
 'Presents - ST2S',
 'Presents - TMD',
 'Presents - STHR',
 'Taux de reussite - STG',
 'Taux de reussite - STI2D',
 'Taux de reussite - STD2A',
 'Taux de reussite - STMG',
 'Taux de reussite - STI',
 'Taux de reussite - STL',
 'Taux de reussite - ST2S',
 'Taux de reussite - TMD',
 'Taux de reussite - STHR',
 'Taux de mentions - STI2D',
 'Taux de mentions - STD2A',
 'Taux de mentions - STMG',
 'Taux de mentions - STL',
 'Taux de mentions - ST2S',
 'Taux de mentions - TMD',
 'Taux de mentions - STHR']]  
df_ayuda=df_ayuda.fillna(0)

df_bac['total_eleves_technique']=df_ayuda['Presents - STG']+df_ayuda['Presents - STI2D']+df_ayuda['Presents - STD2A']+df_ayuda['Presents - STMG']+df_ayuda['Presents - STI']+df_ayuda['Presents - STL']+df_ayuda['Presents - ST2S']+df_ayuda['Presents - TMD']+df_ayuda['Presents - STHR']
df_bac['Taux_reussite_technique']= (df_ayuda['Presents - STG']*df_ayuda['Taux de reussite - STG']+ df_ayuda['Presents - STI2D']*df_ayuda['Taux de reussite - STI2D']+df_ayuda['Presents - STD2A']*df_ayuda['Taux de reussite - STD2A']+df_ayuda['Presents - STMG']*df_ayuda['Taux de reussite - STMG']+df_ayuda['Presents - STI']*df_ayuda['Taux de reussite - STI']+df_ayuda['Presents - STL']*df_ayuda['Taux de reussite - STL']+df_ayuda['Presents - ST2S']*df_ayuda['Taux de reussite - ST2S']+df_ayuda['Presents - TMD']*df_ayuda['Taux de reussite - TMD']+df_ayuda['Presents - STHR']*df_ayuda['Taux de reussite - STHR'])/df_bac['total_eleves_technique']
df_bac['Taux_mentions_technique']= (df_ayuda['Presents - STI2D']*df_ayuda['Taux de mentions - STI2D']+df_ayuda['Presents - STD2A']*df_ayuda['Taux de mentions - STD2A']+df_ayuda['Presents - STMG']*df_ayuda['Taux de mentions - STMG']+df_ayuda['Presents - STL']*df_ayuda['Taux de mentions - STL']+df_ayuda['Presents - ST2S']*df_ayuda['Taux de mentions - ST2S']+df_ayuda['Presents - TMD']*df_ayuda['Taux de mentions - TMD']+df_ayuda['Presents - STHR']*df_ayuda['Taux de mentions - STHR'])/(df_bac['total_eleves_technique']-df_ayuda['Presents - STI']-df_ayuda['Presents - STG'])

df_bac['STG']=np.where(df_bac["Presents - STG"]>0, 1,0)
df_bac['STI2D']=np.where(df_bac["Presents - STI2D"]>0, 1,0)
df_bac['STD2A']=np.where(df_bac["Presents - STD2A"]>0, 1,0)
df_bac['STMG']=np.where(df_bac["Presents - STMG"]>0, 1,0)
df_bac['STI']=np.where(df_bac["Presents - STI"]>0, 1,0)
df_bac['STL']=np.where(df_bac["Presents - STL"]>0, 1,0)
df_bac['ST2S']=np.where(df_bac["Presents - ST2S"]>0, 1,0)
df_bac['TMD']=np.where(df_bac["Presents - TMD"]>0, 1,0)
df_bac['STHR']=np.where(df_bac["Presents - STHR"]>0, 1,0)

df_bac['Nombre_section_tecnique']=df_bac['STG']+df_bac['STI2D']+df_bac['STD2A']+df_bac['STMG']+df_bac['STI']+df_bac['STL']+df_bac['ST2S']+df_bac['TMD']+df_bac['STHR']

In [65]:
df1_bac=df_bac[df_bac['Region'].isin(['Ile-de-France']) & df_bac['code_departement'].isin(['75','92','93','94'])]

In [66]:
df1_bac=df1_bac[["Taux_mentions_technique", "Taux_mentions_general","total_eleves_general","total_eleves_technique","Nombre_section_tecnique", "Taux_reussite_general", "Taux_reussite_technique", "Etablissement","Annee","Ville","UAI","Code commune","Academie","Departement","Secteur","Presents - L","Presents - ES","Presents - S","Presents - STG","Presents - STI2D","Presents - STD2A","Presents - STMG","Presents - STI","Presents - STL","Presents - ST2S","Presents - TMD","Presents - STHR","Presents - Toutes series","Taux de reussite - L","Taux de reussite - ES","Taux de reussite - S","Taux de reussite - STG","Taux de reussite - STI2D","Taux de reussite - STD2A","Taux de reussite - STMG","Taux de reussite - STI","Taux de reussite - STL","Taux de reussite - ST2S","Taux de reussite - TMD","Taux de reussite - STHR","Taux de reussite - Toutes series","Taux de reussite attendu france - L","Taux de reussite attendu france - ES","Taux de reussite attendu france - S","Taux de reussite attendu france - STG","Taux de reussite attendu france - STI2D","Taux de reussite attendu france - STD2A","Taux de reussite attendu france - STMG","Taux de reussite attendu france - STI","Taux de reussite attendu france - STL","Taux de reussite attendu france - ST2S","Taux de reussite attendu france - TMD","Taux de reussite attendu france - STHR","Taux de reussite attendu france - Toutes series","Taux de mentions - L","Taux de mentions - ES","Taux de mentions - S","Taux de mentions - STI2D","Taux de mentions - STD2A","Taux de mentions - STMG","Taux de mentions - STL","Taux de mentions - ST2S","Taux de mentions - TMD","Taux de mentions - STHR","Taux de mentions - Toutes series","Taux de mentions attendu - L","Taux de mentions attendu - ES","Taux de mentions attendu - S","Taux de mentions attendu - STI2D","Taux de mentions attendu - STD2A","Taux de mentions attendu - STMG","Taux de mentions attendu - STL","Taux de mentions attendu - ST2S","Taux de mentions attendu - TMD","Taux de mentions attendu - STHR","Taux de mentions attendu - Toutes series","Structure pedagogique 5 groupes","Structure pedagogique 7 groupes","Effectif de seconde","Effectif de premiere","Effectif de terminale","Taux d'acces 2nde-bac","Taux d'acces attendu france 2nde-bac","Taux d'acces 1ere-bac","Taux d'acces attendu france 1ere-bac","Taux d'acces terminale-bac", "Taux d'acces attendu france terminale-bac","Region","Code region","code_departement"]]

In [68]:
cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256)

# Partie 1. Analyse du taux de réussite de Paris 2020 

## 1.1 BAC

## 1.1.1 Nombre d'élève par département

In [69]:
fig = px.histogram(df1_bac[df1_bac['Annee']==2020].groupby(['Annee','code_departement'])['total_eleves_technique'].sum().reset_index(), x='code_departement',y="total_eleves_technique", title='Total eleves - TECHNIQUE', width=950, height=560)
fig.update_layout(legend =dict(font = dict(family = "Courier", size = 10, color = "black")),
                  legend_title = dict(font = dict(size = 20, color = "blue")))
fig.show()

In [70]:
fig = px.histogram(df1_bac[df1_bac['Annee']==2019].groupby(['Annee','code_departement'])['total_eleves_general'].sum().reset_index(), x='code_departement',y="total_eleves_general", title='Total eleves - GENERAL', width=950, height=560)
fig.update_layout(legend =dict(font = dict(family = "Courier", size = 10, color = "black")),
                  legend_title = dict(font = dict(size = 20, color = "blue")))
fig.show()

On constate qu'il existe une différence dans la proportion du nombre d'étudiants en section générale et technique entre les départements. Nous avons plus d'étudiants en section générale en 75 et plus d'étudiants en section technique en 93.

## 1.1.2 Nombre d'élève par commune

In [71]:
df2_bac=df1_bac[df1_bac['Academie']=='PARIS']
fig = px.histogram(df2_bac[df2_bac['Annee']==2019].groupby(['Annee','Ville'])['total_eleves_technique'].sum().reset_index(), x='Ville',y="total_eleves_technique", title='Total eleves - TECHNIQUE', width=950, height=560)
fig.update_layout(legend =dict(font = dict(family = "Courier", size = 10, color = "black")),
                  legend_title = dict(font = dict(size = 20, color = "blue")))
fig.show()

In [72]:
df2_bac=df1_bac[df1_bac['Academie']=='PARIS']
fig = px.histogram(df2_bac[df2_bac['Annee']==2019].groupby(['Annee','Ville'])['total_eleves_general'].sum().reset_index(), x='Ville',y="total_eleves_general", title='Total eleves - GENERAL', width=950, height=560)
fig.update_layout(legend =dict(font = dict(family = "Courier", size = 10, color = "black")),
                  legend_title = dict(font = dict(size = 20, color = "blue")))
fig.show()

LORETO ECRIRE**************

In [73]:
df_bac.isnull().sum().head(30)

Etablissement                   0
Annee                           0
Ville                           0
UAI                             0
Code commune                    0
Academie                        0
Departement                     0
Secteur                         0
Presents - L                 4732
Presents - ES                3030
Presents - S                 1866
Presents - STG              18360
Presents - STI2D            16052
Presents - STD2A            20132
Presents - STMG             12142
Presents - STI              20215
Presents - STL              18520
Presents - ST2S             16851
Presents - TMD              20632
Presents - STHR             20239
Presents - Toutes series        0
Taux de reussite - L         4732
Taux de reussite - ES        3030
Taux de reussite - S         1866
Taux de reussite - STG      18360
Taux de reussite - STI2D    16052
Taux de reussite - STD2A    20132
Taux de reussite - STMG     12142
Taux de reussite - STI      20215
Taux de reussi

Parce que certains parcours tel section STD2A, section STG.. etc. que n'existent pas dans les lycées étudiés, donc il a l'air logique qu'on ait des valeurs manquantes dans ces sections.

In [74]:
# BAC général
df1=df_bac.loc[df_bac['Academie'] == 'PARIS']
df2=df1[['Etablissement','Annee','Ville','Academie','Taux de reussite - L','Taux de reussite - ES','Taux de reussite - S']]

In [99]:
df3 = df2[['Etablissement','Annee','Ville','Academie','Taux de reussite - L','Taux de reussite - ES','Taux de reussite - S']]
df3=df2.loc[df2['Annee'] == 2018.0]

In [100]:
df3.groupby(['Ville','Annee']).agg(avg_tx_réussite_L=('Taux de reussite - L','mean'),avg_tx_réussite_ES = ('Taux de reussite - ES', 'mean'),avg_tx_réussite_S = ('Taux de reussite - S', 'mean'))

,,avg_tx_réussite_L,avg_tx_réussite_ES,avg_tx_réussite_S
Ville,Annee,,,
PARIS 03,2018,95.666667,95.500000,93.000000
PARIS 04,2018,99.500000,98.750000,99.500000
PARIS 05,2018,100.000000,100.000000,99.800000
PARIS 06,2018,98.285714,97.714286,98.500000
PARIS 07,2018,95.750000,98.000000,99.833333
PARIS 08,2018,99.500000,99.666667,98.333333
PARIS 09,2018,93.200000,93.000000,88.600000
PARIS 10,2018,98.250000,96.000000,95.250000
PARIS 11,2018,94.000000,84.000000,91.250000


In [ ]:
qingyi

In [96]:
# BAC technique
df2a = df1[['Etablissement','Annee','Ville','Academie','Presents - STG',
 'Presents - STI2D',
 'Presents - STD2A',
 'Presents - STMG',
 'Presents - STI',
 'Presents - STL',
 'Presents - ST2S',
 'Presents - TMD',
 'Presents - STHR',
 'Taux de reussite - STG',
 'Taux de reussite - STI2D',
 'Taux de reussite - STD2A',
 'Taux de reussite - STMG',
 'Taux de reussite - STI',
 'Taux de reussite - STL',
 'Taux de reussite - ST2S',
 'Taux de reussite - TMD',
 'Taux de reussite - STHR',
 'Taux de mentions - STI2D',
 'Taux de mentions - STD2A',
 'Taux de mentions - STMG',
 'Taux de mentions - STL',
 'Taux de mentions - ST2S',
 'Taux de mentions - TMD',
 'Taux de mentions - STHR']]
df4=df2a.loc[df2['Annee'] == 2018.0]

In [97]:
df_ayuda=df4[[
 'Presents - STG',
 'Presents - STI2D',
 'Presents - STD2A',
 'Presents - STMG',
 'Presents - STI',
 'Presents - STL',
 'Presents - ST2S',
 'Presents - TMD',
 'Presents - STHR',
 'Taux de reussite - STG',
 'Taux de reussite - STI2D',
 'Taux de reussite - STD2A',
 'Taux de reussite - STMG',
 'Taux de reussite - STI',
 'Taux de reussite - STL',
 'Taux de reussite - ST2S',
 'Taux de reussite - TMD',
 'Taux de reussite - STHR',
 'Taux de mentions - STI2D',
 'Taux de mentions - STD2A',
 'Taux de mentions - STMG',
 'Taux de mentions - STL',
 'Taux de mentions - ST2S',
 'Taux de mentions - TMD',
 'Taux de mentions - STHR']]  
df_ayuda=df_ayuda.fillna(0)
df4['total_eleves_technique']=df_ayuda['Presents - STG']+df_ayuda['Presents - STI2D']+df_ayuda['Presents - STD2A']+df_ayuda['Presents - STMG']+df_ayuda['Presents - STI']+df_ayuda['Presents - STL']+df_ayuda['Presents - ST2S']+df_ayuda['Presents - TMD']+df_ayuda['Presents - STHR']
df4['Taux_reussite_technique']= (df_ayuda['Presents - STG']*df_ayuda['Taux de reussite - STG']+ df_ayuda['Presents - STI2D']*df_ayuda['Taux de reussite - STI2D']+df_ayuda['Presents - STD2A']*df_ayuda['Taux de reussite - STD2A']+df_ayuda['Presents - STMG']*df_ayuda['Taux de reussite - STMG']+df_ayuda['Presents - STI']*df_ayuda['Taux de reussite - STI']+df_ayuda['Presents - STL']*df_ayuda['Taux de reussite - STL']+df_ayuda['Presents - ST2S']*df_ayuda['Taux de reussite - ST2S']+df_ayuda['Presents - TMD']*df_ayuda['Taux de reussite - TMD']+df_ayuda['Presents - STHR']*df_ayuda['Taux de reussite - STHR'])/df4['total_eleves_technique']

In [98]:
df4.groupby(['Ville','Annee']).agg(avg_tx_réussite_tech= ('Taux_reussite_technique','mean'))

,,avg_tx_réussite_tech
Ville,Annee,
PARIS 03,2018,91.500000
PARIS 04,2018,100.000000
PARIS 05,2018,97.000000
PARIS 06,2018,99.040309
PARIS 07,2018,98.509677
PARIS 08,2018,98.000000
PARIS 09,2018,82.500000
PARIS 10,2018,NaN
PARIS 11,2018,95.500000


On observe que les 4e，6e，7e, 8e, 13e arrondissements de Paris sont très performants avec le note moyen supérieur à 95 en l'année 2018 et par contre que les moins performants sont les 9e, 14e et 15e arrondissements de Paris, ils ont eu un note moyen inférieur à 90 en l'année 2018.

## Pour les données socio-economiques

In [102]:
#Base de données pour l'analyse de CSP
#SOURCE: https://www.insee.fr/fr/statistiques/5395838?sommaire=5395900#consulter
idf= pd.read_excel('Ale/Donn/soceco2018.xlsx')

In [103]:
# Le taux de chomage 
# Tous 15-64
idf['TCHOM1564']= 100*idf['P18_CHOM1564'] / idf['P18_ACT1564']

In [104]:
# Part de cadres dans la population active
idf['TCADACTOCC1564'] = 100*idf['C18_ACTOCC1564_CS3'] / idf['C18_ACTOCC1564']

In [105]:
# Le taux d'emploi 
# Tous 15-64
idf['TEMPL1564']= 100*idf['P18_ACTOCC1564'] / idf['P18_POP1564']

In [106]:
# On vien de computer les statistiques pour les autres categories socioprofessionnelles.
# Part d'artisans, commercants et chefs d'entreprise dans la population active
idf['TACCACTOCC1564'] = 100*idf['C18_ACTOCC1564_CS2'] / idf['C18_ACTOCC1564']

In [107]:
# Part de professionnelles intermediares dans la population active occupée.
idf['TINTACTOCC1564'] = 100*idf['C18_ACTOCC1564_CS4'] / idf['C18_ACTOCC1564']

In [108]:
# Part d'employés dans la population active occupée.
idf['TEMPLACTOCC1564'] = 100*idf['C18_ACTOCC1564_CS5'] / idf['C18_ACTOCC1564']

In [109]:
# Part d'ouvriers dans la population active occupée.
idf['TOUVRACTOCC1564'] = 100*idf['C18_ACTOCC1564_CS6'] / idf['C18_ACTOCC1564']

### Pour une meilleure visualisation et computation des statistiques descriptives on selectionne que les variables qu'on veut observer.

In [110]:
idf= idf[['CODGEO','DEP','LIBGEO','TEMPL1564','TCHOM1564','TCADACTOCC1564','TACCACTOCC1564','TINTACTOCC1564','TEMPLACTOCC1564',
         'TOUVRACTOCC1564']]
# On selectionne que les colomnes que on utiliserà 

In [115]:
idf.groupby('DEP')['TEMPL1564','TCHOM1564','TCADACTOCC1564','TACCACTOCC1564','TINTACTOCC1564','TEMPLACTOCC1564',
         'TOUVRACTOCC1564'].agg(['mean','std']).round(2)
# On montre le moyenne et la deviation standard pour les données

TEMPL1564       TCHOM1564       TCADACTOCC1564        TACCACTOCC1564  \
         mean   std      mean   std           mean    std           mean   
DEP                                                                        
75      69.78  3.68     10.84  1.87          50.65   5.47           6.65   
92      70.01  4.14     10.49  2.79          42.47  10.42           5.15   
93      60.95  6.19     17.03  4.35          16.78   8.19           5.46   
94      67.32  4.31     11.70  3.20          26.75  10.68           5.48   

          TINTACTOCC1564       TEMPLACTOCC1564       TOUVRACTOCC1564        
      std           mean   std            mean   std            mean   std  
DEP                                                                         
75   2.12          21.08  2.65           16.41  3.12            5.18  1.74  
92   1.63          23.89  2.37           20.86  5.98            7.61  4.07  
93   1.04          25.20  4.07           32.61  5.51           19.93  6.36  
94   1.73          27.16  2.79           27.37  5.91           13.19  5.78

Le taux d'emploi est plus élevé dans le haut de seine comme le taux de chommage que dans ce cas est le plus bas en moyenne.
Dans Paris il y a moins d'ouvriers ou employées que dans la petite couronne, en particulier si on regarde la difference entre Paris et le departements 93/94 la difference est plus elevée. Ce constat est aussi observable dans le fait que il y a plus de cadres a Paris et dans le Hauts-de-Seine que dans le departements de Seine-Saint-Denis et Val-de-Marne.

In [114]:
idf.loc[idf['DEP'] == 75].groupby('CODGEO')['TEMPL1564','TCHOM1564','TCADACTOCC1564','TACCACTOCC1564','TINTACTOCC1564','TEMPLACTOCC1564',
         'TOUVRACTOCC1564'].agg(['mean']).round(2)

,TEMPL1564,TCHOM1564,TCADACTOCC1564,TACCACTOCC1564,TINTACTOCC1564,TEMPLACTOCC1564,TOUVRACTOCC1564
,mean,mean,mean,mean,mean,mean,mean
CODGEO,,,,,,,
75101,70.41,10.65,51.09,9.33,19.83,15.34,4.41
75102,76.80,9.79,53.83,7.93,19.79,13.42,4.99
75103,74.05,9.96,52.60,8.07,21.97,13.86,3.50
75104,70.60,11.09,50.92,7.70,22.06,15.56,3.69
75105,65.00,9.15,57.44,6.38,19.62,13.04,3.49
75106,62.97,9.62,55.75,9.26,17.60,13.44,3.86
75107,68.32,9.01,54.25,9.73,16.87,15.27,3.80
75108,70.80,8.23,53.14,9.46,16.89,15.66,4.78


Pour Paris, le taux de chommage est plus élevé dans le 19ème et 20ème arrondissements mais le taux d'emploi est plus bas dans le 6ème. Pour ce que regarde les catégories socio-professionnelles les cadres se situent hors 19ème et 20ème arrondissements que sont aussi les arrondissements avec le plus d'employées.